# "Road to recovery"

Simulation of various control measures under the asumption of remaining exposed (infected but not infectious) within the community.Arbitrayr epidemic parameters are presumed and then model with different network structures which accurately reflect different levels of "work-from-home", restrictions on gatherings, and/or COVIDSafe adoption - *under the assumption of those arbitrary choces of epidemic parameters*.

First, load some libraries....

In [39]:
cd("/Users/michael/work/GitHub/epinets")

In [40]:
include("EpiSim.jl")

Main.EpiSim

In [41]:
using CSV
using Plots
using LightGraphs
using JLD2, FileIO
using Statistics

Population of WA

In [42]:
pops=[426709 8089526 245869 5095100 1751693 534281 6594804 2621680]
pop=pops[8]

2621680

## Parameter guesstimates. 

Note, unlike previous simulations we only care about one choice of each parameter - there is no changepoint here.

In [43]:
#'reasonable' parameters
epiparam=Dict()
epiparam["p0"]=1/8 #a guess - tuned to match observed data 
epiparam["q"]=1/7 #"up to" two weeks
epiparam["r0"]=1/6 #about two weeks for mild, 3-6 for severe
epiparam["nseeds"]=5 #probably too many, consider dropping.
#parameters don't change across transition point - assume about 5 undiagnosed case 
#and control the covidsafe uptake via network structure
i=8
epiparam["pop"]=Int(floor(sqrt(pops[i])))^2
epiparam["gridsize"]=Int(floor(sqrt(pops[i])))

1619

## Build sample contact networks


Build contact networks with no restrictions and mass gatherings (bamodel), purely local (lattice), some level of work-from-home (wattsrog), or random mixing but no mass gatherings (randomgraph)

In [66]:
gridsize=epiparam["gridsize"]
bamodel=barabasi_albert(gridsize^2, 3, 2)
lattice=LightGraphs.grid((gridsize,gridsize),periodic=true)
#"social distancing"
wattstrog95=watts_strogatz(gridsize^2, 4, 0.013)  #s=0.013 => 95% compliance
wattstrog90=watts_strogatz(gridsize^2, 4, 0.026)  #s=0.026 => 90% compliance
wattstrog80=watts_strogatz(gridsize^2, 4, 0.053)  #s=0.053 => 80% compliance
wattstrog60=watts_strogatz(gridsize^2, 4, 0.120)  #s=0.120 => 60% compliance
wattstrog40=watts_strogatz(gridsize^2, 4, 0.205)  #s=0.205 => 40% compliance
wattstrog20=watts_strogatz(gridsize^2, 4, 0.332)  #s=0.332 => 20% compliance
randomgraph=watts_strogatz(gridsize^2, 4, 1)  #0% compliance

{2621161, 5242322} undirected simple Int64 graph

To simulate the effect of COVIDSafe, we remove links corresponding to the contacts that would be traced with the app. That is, if k is the fraction with the app, then we deletec k^2 edges from our graph.

In [67]:
#"covidsafe" at 40%
covidsafe=0.4
iso=covidsafe^2 #covidsafe implies removing iso fraction of all edges of graph
covidsafe40=deepcopy(randomgraph) 
for edg in edges(covidsafe40)
    if rand(Float64) .< iso
        rem_edge!(covidsafe40,edg)
    end
end
#there are two parameters to play with here - the underlying model (here bamodel, but it could be an ER graph, or a truncated
#scale free network, or a lattice - wattsrog - with a certain fraction of remote links) The second parameter is the level of covidsafe adoption
# of course the original transmission parameters could be changed too....

In [68]:
#"covidsafe" at 60%
covidsafe=0.6
iso=covidsafe^2 #covidsafe implies removing iso fraction of all edges of graph
covidsafe60=deepcopy(wattstrog40) 
for edg in edges(covidsafe60)
    if rand(Float64) .< iso
        rem_edge!(covidsafe60,edg)
    end
end
#40% work from home (wattstrog40 plus 60% COVIDSafe)

In [69]:
#"covidsafe" at 80%
covidsafe=0.8
iso=covidsafe^2 #covidsafe implies removing iso fraction of all edges of graph
covidsafe80=deepcopy(bamodel) 
for edg in edges(covidsafe80)
    if rand(Float64) .< iso
        rem_edge!(covidsafe80,edg)
    end
end
#40% work from home (wattstrog40 plus 60% COVIDSafe)

## Simulations

Now we run a bunch of simualtions with these presumed control parameters and the network contact graphs generated above

In [76]:
ndays=150#120 days, 100 simulations
nsims=100
St0,Et0,It0,Rt0=EpiSim.episim(bamodel, epiparam, ndays, nsims) 
St1,Et1,It1,Rt1=EpiSim.episim(covidsafe80, epiparam, ndays, nsims)
St2,Et2,It2,Rt2=EpiSim.episim(randomgraph, epiparam, ndays, nsims) 
St3,Et3,It3,Rt3=EpiSim.episim(covidsafe40, epiparam, ndays, nsims) 
St4,Et4,It4,Rt4=EpiSim.episim(wattstrog40, epiparam, ndays, nsims) 
St5,Et5,It5,Rt5=EpiSim.episim(covidsafe60, epiparam, ndays, nsims) 

100.0%┣████████████████████████████████████████┫ 100/100 [24:46<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [18:07<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [42:34<00:00, 0.0 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [20:07<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [29:47<00:00, 0.1 it/s]
100.0%┣████████████████████████████████████████┫ 100/100 [16:06<00:00, 0.1 it/s]


(UInt64[0x000000000027fee4 0x000000000027fee4 … 0x000000000027fee4 0x000000000027fee4; 0x000000000027fee4 0x000000000027fee4 … 0x000000000027fee4 0x000000000027fee4; … ; 0x000000000027feb6 0x000000000027fea4 … 0x000000000027fed7 0x000000000027fed0; 0x000000000027feb6 0x000000000027fea4 … 0x000000000027fed7 0x000000000027fed0], UInt64[0x0000000000000004 0x0000000000000004 … 0x0000000000000004 0x0000000000000004; 0x0000000000000004 0x0000000000000004 … 0x0000000000000004 0x0000000000000003; … ; 0x0000000000000001 0x0000000000000000 … 0x0000000000000000 0x0000000000000000; 0x0000000000000001 0x0000000000000000 … 0x0000000000000000 0x0000000000000000], UInt64[0x0000000000000001 0x0000000000000001 … 0x0000000000000001 0x0000000000000001; 0x0000000000000001 0x0000000000000001 … 0x0000000000000001 0x0000000000000001; … ; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x0000000000000000; 0x0000000000000000 0x0000000000000000 … 0x0000000000000000 0x0000000000000000], UInt64[0x000000

## Plot

In [32]:
compl=0.2
swtch=1-compl^(1/4)
plotly()

0.331259695023578

In [81]:
plot([],title="Total anticipated infections",xlabel="day",ylabel="Infections",ylimit=(0,5000),label=false,size=(1200,800))
EpiSim.plotquantiles(pop .- St0,:black,"Back to the footy")
EpiSim.plotquantiles(pop .- St1,:red,"Footy & 80% COVIDSafe")
EpiSim.plotquantiles(pop .- St2,:yellow,"No mass gatherings")
EpiSim.plotquantiles(pop .- St3,:orange,"40% COVIDSafe")
EpiSim.plotquantiles(pop .- St4,:blue,"40% Work-from-home")
EpiSim.plotquantiles(pop .- St5,:green,"40% Work-from-home & 60% COVIDSafe")

<!DOCTYPE html>
 
 
 Plots.jl

In [83]:
plot([],title="Current anticipated infections",xlabel="day",ylabel="Infections",ylimit=(0,500),label=false,size=(1200,800))
EpiSim.plotquantiles(It0,:black,"Back to the footy")
EpiSim.plotquantiles(It1,:red,"Footy & 80% COVIDSafe")
EpiSim.plotquantiles(It2,:yellow,"No mass gatherings")
EpiSim.plotquantiles(It3,:orange,"40% COVIDSafe")
EpiSim.plotquantiles(It4,:blue,"40% Work-from-home")
EpiSim.plotquantiles(It5,:green,"40% Work-from-home & 60% COVIDSafe")

<!DOCTYPE html>
 
 
 Plots.jl